In [4]:
from tkinter import *
from tkinter import ttk
import cv2
import numpy as np
from datetime import date, datetime, time, timedelta
from PIL import Image, ImageTk

In [5]:
root = Tk()
root.rowconfigure(0,weight=1)
root.columnconfigure(0,weight=1)
root.geometry("700x500")

root.title("Traffic Interface")

mainframe = ttk.Frame(root)
mainframe.grid(sticky=[N, S, E, W])

left_panel = ttk.Frame(mainframe)
left_panel.grid(row=0,column=0)

right_panel = ttk.Frame(mainframe, relief="raised")
right_panel.grid(row=0, column=1)


canvas = Canvas(right_panel, width=300, height=300)  
canvas.pack(fill="both", expand=True)
canvas.update()

# ret, frame = cap.read()
# if not ret:
#     print("Can't receive frame (stream end?). Exiting ...")
print(canvas.winfo_width(), canvas.winfo_height())
img = ImageTk.PhotoImage(Image.fromarray(test_frame).resize((mainframe.winfo_width(), mainframe.winfo_height())))

print(canvas.winfo_width(), canvas.winfo_height())

label = Label(canvas, image=img).grid(row=0, column=0, sticky=E)
# label.pack()

for i in range(3):
    for j in range(3):
        frame = ttk.Frame(
            master=left_panel,
            relief="raised",
            borderwidth=1
        )
        frame.grid(row=i, column=j)
        label = ttk.Label(master=frame, text=f"Row {i}\nColumn {j}")
        label.pack()

root.mainloop()

304 304


NameError: name 'test_frame' is not defined

In [2]:
source = "vids\TLC00011.mp4"
cap = cv2.VideoCapture(source)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

dim = cap.read()[1].shape

AttributeError: 'NoneType' object has no attribute 'shape'

In [9]:
dim

(720, 1280, 3)

In [6]:
nums_y_loc = 704
num_height = 13
num_width = 13

# x pixel of 10s and 1s digit respectively
hours_x = 752, 768 
mins_x = 800, 816
secs_x = 848, 864

years_x = 576, 592, 608, 624
months_x = 656, 672
days_x = 704, 720

numRef = {
    '[10, 10, 13, 13, 7, 7, 7, 7, 7, 7, 13, 13, 10]': 0,
    '[0, 0, 1, 1, 3, 3, 13, 13, 13, 13, 1, 1, 1]':  1,
    '[5, 5, 9, 9, 5, 5, 7, 7, 5, 5, 11, 11, 7]': 2,
    '[4, 4, 7, 7, 3, 3, 5, 5, 5, 5, 13, 13, 8]' : 3,
    '[4, 4, 6, 6, 6, 6, 6, 6, 13, 13, 13, 13, 2]' : 4,
    '[8, 8, 9, 9, 5, 5, 5, 5, 5, 5, 11, 11, 8]': 5,
    '[8, 8, 11, 11, 7, 7, 5, 5, 5, 5, 9, 9, 4]': 6,
    '[4, 4, 4, 4, 7, 7, 9, 9, 6, 6, 6, 6, 4]': 7, 
    '[8, 8, 13, 13, 5, 5, 5, 5, 5, 5, 13, 13, 8]': 8,
    '[4, 4, 9, 9, 5, 5, 5, 5, 7, 7, 12, 12, 8]': 9,
}


In [4]:
def showFrame(frame, show, title="Unnamed Frame"):
    """Show the frame in question if show is true for debugging.

    If show is false, does nothing.
    If show is true, shows frame as an external window until a keypress is recorded.

    Args:
        frame: frame to show.
        title: title of the frame to show. Defaults to "Unnamed Frame"
        show: determines whether to show or to skip

    Returns:
        Nothing

    Raises:
    """
    
    if (show):
        cv2.imshow(title, frame)  # show frame on window
        cv2.waitKey(0)

        # closing all open windows
        cv2.destroyAllWindows()

def getFrame(frame_num, cap=cap, show=False):
    """Retrieves the frame from the video

    Retrieve the indexed frame from the video and return it as an image of an numpy array.

    Args:
        frame_num: index of the frame to show.
        cap: capture to find the frame of, defaults to cap defined above
        show: determines whether to show or to skip, defaults to noshow

    Returns:
        numpy image array of the frame

    Raises:
        Exception: frame_num({frame_num}) cannot equal or exceed number of frames({frame_count})
    """
    
    if (frame_num >= frame_count): 
        raise Exception(f"frame_num({frame_num}) cannot equal or exceed number of frames({frame_count})") 
    
    cap.set(1,frame_num)  # Where frame_no is the frame you want
    ret, frame = cap.read()  # Read the frame
    
    showFrame(frame, show, title="frame")
    
    return frame

def getNum(frame, xcoord, ycoord=nums_y_loc, show=False):
    """Retrieves a number from the video

    Identify the number from a coordinate specified in the video and return it as a integer.

    Args:
        frame: numpy array image to identify.
        xcoord: x coordinate of the top-left pixel of the number
        ycoord: y coordinate of the top-left pixel, defaults to the nums_y_loc
        show: determines whether to show or to skip, defaults to noshow

    Returns:
        integer at that location, -1 if invalid
    """
    print("get num start")
    gray = np.around(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) / 255)
    print("to gray")
    num_crop = gray[ycoord : ycoord + num_width, xcoord : xcoord + num_width]
    print("cropped")
    
    showFrame(frame, show, title="cropped number")
        
    hashed = str(list(sum(num_crop))).replace(".0", "")
    
    if (hashed not in numRef):
        return -1
        
    return numRef[hashed]

def getDate(frame, show=False):
    years = 1000 * getNum(frame, years_x[0]) + 100 * getNum(frame, years_x[1]) + 10 * getNum(frame, years_x[2]) +  getNum(frame, years_x[3])
    months = 10 * getNum(frame, months_x[0]) + getNum(frame, months_x[1])
    days = 10 * getNum(frame, days_x[0]) + getNum(frame, days_x[1])
    
    dateCrop = frame[nums_y_loc : nums_y_loc + 13, years_x[0] : days_x[1] + 13]
    showFrame(dateCrop, show, title="cropped date")
    
    return datetime(years, months, days)
    

def getTime(frame, show=False):
    hours = 10 * getNum(frame, hours_x[0]) + getNum(frame, hours_x[1])
    mins = 10 * getNum(frame, mins_x[0]) + getNum(frame, mins_x[1])
    secs = 10 * getNum(frame, secs_x[0]) + getNum(frame, secs_x[1])
    
    dateCrop = frame[nums_y_loc : nums_y_loc + 13, hours_x[0] : secs_x[1] + 13]
    showFrame(dateCrop, show, title="cropped time")
    
    return datetime.combine(datetime.today(), time(hours, mins, secs))

def getDateTime(frame, show=False):
    dateCrop = frame[nums_y_loc : nums_y_loc + 13, years_x[0] : secs_x[1] + 13]
    showFrame(dateCrop, show, title="cropped datetime")
    
    return datetime.combine(getDate(frame), getTime(frame).time())

def getFPS(cap=cap):
    init_time = getTime(getFrame(0))
    j = 0;
    while j < frame_count and getTime(getFrame(j)) == init_time: j += 1
        
    secCounter = 0
    i = j
     # start of the first sec
    new_time = getTime(getFrame(j))
    while secCounter < 4:
        while i < frame_count and getTime(getFrame(i)) == new_time: i += 1 # start of the second second
        new_time = getTime(getFrame(i))
        secCounter += 1
    
    return (i - j)/4

fps = 5 #getFPS()

def getAccurateIndex(index, time, fps=fps, cap=cap):
    while (getDateTime(getFrame(index)) >= time):
        index -= 1
    
    while (getDateTime(getFrame(index)) < time):
        index += 1
    
    return index

def getFrameIndexOfTime(time, fps=fps, cap=cap):
    init_time = getDateTime(getFrame(0))
    time_delt = time - init_time
    
    if (time_delt.total_seconds() < 0): return -1
    
    jump_index = time_delt.total_seconds() * fps
    return getAccurateIndex(jump_index, time)

def getFrameAfterSeconds(secs, fps=fps, cap=cap):
    return getFrameIndexOfTime(time)

In [5]:
test_frame = getFrame(0)

In [8]:
fps

5.0

In [6]:
# getFPS()
# init = getDateTime(getFrame(0), show=True)
# later = getDateTime(getFrame(500), show=True)

# print(init)
# print(later)
# print(later - init)
# print(fps)
# print(fps * (later - init).total_seconds())

# getFrame(500, show=True)
# getDate(getFrame(500))
# jump_index = getFrameIndexOfTime(getDateTime(getFrame(2539)))
print(jump_index)
# getSecsElapsed()

2539.0


In [3]:
import tkinter as tk
root = tk.Tk()
w = tk.Entry(root)
w.pack()
root.mainloop()

In [5]:
import tkinter as tk

root = tk.Tk()
root.geometry("300x200")

def func(event):
    print("You hit return.")

def onclick(event):
    print("You clicked the button")

root.bind('<Return>', onclick)

button = tk.Button(root, text="click me")
button.bind('<Button-1>', onclick)
button.pack()

root.mainloop()

You clicked the button
You clicked the button
You clicked the button


In [23]:
from tkinter import *

# def keydown(event):
#     if not keyDown:
#         print(event.keysym, event.keysym=='keypressed')
#         print(event)
#         keyDown = True
    
def keyup(event):
    keyDown = False
    print(event.keysym)
    print(event)

root = Tk()

keyDown = True
# root.bind("<KeyPress>", keydown)
root.bind("<KeyRelease>", keyup)
root.mainloop()

d
<KeyRelease event state=Mod1 keysym=d keycode=68 char='d' x=106 y=127>


In [62]:
import tkinter as tk
root = tk.Tk()
testFrame = tk.Frame(root)
testFrame.pack(fill=tk.BOTH, expand=True)

testFrame.rowconfigure(0,weight=1)
testFrame.columnconfigure(0,weight=1)
# testFrame.columnconfigure(1,weight=10)



b = tk.Button(testFrame, text="Frame: ")

def callback(P):
    if str.isdigit(P) or P == "":
        return True
    else:
        return False
    
w = tk.Entry(testFrame, width=5, validate='all', validatecommand=(testFrame.register(callback), '%P')) 

b.grid(row=0, column=0, sticky="nsew")
w.grid(row=0, column=1, sticky="nsew")
root.mainloop()

In [46]:
try:
    print(int('3'))
except:
    print("invalid")

3


In [7]:
import tkinter as tk

class testPanel(tk.Frame):
    def __init__(self, parent):
        super().__init__()
        testlabel1 = tk.Label(self, bg="red")
        testlabel1.grid(row=0, column=0)
        testlabel1.bind("<Button-1>", self.click)
        testlabel0 = tk.Label(self, bg="green")
        testlabel0.grid(row=0, column=0)
        testlabel0.bind("<Button-1>", self.click)
        print("here!")
    
    def click(self, event):
        print(event)

if __name__ == "__main__":
    root =  tk.Tk()
#     root.geometry("500x300")
    
    tp = testPanel(root)
    tp.pack()
    
    root.mainloop()
    

here!
<ButtonPress event state=Mod1 num=1 x=4 y=0>
<ButtonPress event state=Mod1 num=1 x=2 y=10>
<ButtonPress event state=Mod1 num=1 x=2 y=14>
<ButtonPress event state=Mod1 num=1 x=5 y=14>
<ButtonPress event state=Mod1 num=1 x=5 y=16>


In [2]:
import tkinter as tk
from PIL import Image, ImageTk, ImageOps

class testPanel(tk.Frame):
    def __init__(self, parent):
        super().__init__()
        self.testcanvas = tk.Canvas(self, bg="red")
        self.testcanvas.grid(row=0, column=0)
        self.rowconfigure(0, weight=1)
        self.columnconfigure(0, weight=1)
        print(self.testcanvas.winfo_width() // 2, self.testcanvas.winfo_height() // 2)
        self.img = Image.open("..\\map.png")
        
#         self.img.show()
        self.image = ImageTk.PhotoImage(self.img)
        self.curr_img = self.image
        self.create_image = self.testcanvas.create_image(self.testcanvas.winfo_width() // 2, 
                                                         self.testcanvas.winfo_height() // 2,
                                                         image=self.image)
        self.testcanvas.bind("<Button-1>", self.click)
        print("here!")
        
        self.testcanvas.bind("<Configure>", self._resize_image) # make vid frame dynamic
        self.curr_image = None
    
    def click(self, event):
        print(event)
        print (event.x - self.testcanvas.winfo_width() // 2 + self.resized_tk.width() // 2, 
               event.y - self.testcanvas.winfo_height() // 2 + self.resized_tk.height() // 2, )
        
    def _resize_image(self, event):
        print("here!")
        self.testcanvas.update()
        self.vid_frame_dims = event.width, event.height
#             ratio = (self.dims[0] / self.dims[1])
        self.resized_image = ImageOps.contain(self.img.copy(), (self.vid_frame_dims[0], self.vid_frame_dims[1]))
        self.resized_image.resize((self.vid_frame_dims[0], self.vid_frame_dims[1]))
#         self.resized_image.show()
        self.resized_tk = ImageTk.PhotoImage(self.resized_image.convert(mode="RGBA"))
        self.curr_image = self.resized_tk
        self.testcanvas.itemconfig(self.create_image, image=self.resized_tk)
        self.testcanvas.coords(self.create_image, self.vid_frame_dims[0] // 2, self.vid_frame_dims[1] // 2)

if __name__ == "__main__":
    root =  tk.Tk()
#     root.geometry("500x300")

    root.rowconfigure(0,weight=1)
    root.columnconfigure(0,weight=1)
    
    tp = testPanel(root)
    tp.pack()
    
    root.mainloop()
    

0 0
here!
here!
<ButtonPress event state=Mod1 num=1 x=11 y=7>
11 -17
<ButtonPress event state=Mod1 num=1 x=3 y=26>
3 2
<ButtonPress event state=Mod1 num=1 x=366 y=1>
366 -23


In [1]:
import tkinter as tk
from tkinter import Tk, ttk

header = ["Time", "L1", "L2", "L3", "L4", "L5", "L6", "L7"]
position_list = []
position_frame_list = []

fill=tk.BOTH
expand=True

root = Tk()

# input panel
inputBox = ttk.Frame(master=root)
inputBox.pack()

# entry panel
entryBox = ttk.Frame(master=root)
entryBox.pack(side="bottom")

# create input lanes rows 
for i in range(6):
    # create vertical header
    position_frame_list.append(ttk.Frame(
        master=root,
        borderwidth=3
    ))
    position_frame_list[i].pack(fill=fill, expand=expand)

    position_list.append((tk.Label(master=position_frame_list[i], 
                                   text=header[i + 1]),
                          tk.Label(master=position_frame_list[i],
                                   text="None\t"),
                          tk.Button(master=position_frame_list[i],
                                   text="Set")))
    position_list[i][0].pack(fill=fill, expand=expand, side="left")
    position_list[i][1].pack(fill=fill, expand=expand, side="left")
    position_list[i][2].pack(fill=fill, expand=expand, side="left")

# create entry box
frame = ttk.Frame(
            master=root,
            relief="raised",
            borderwidth=1,
        )

frame.pack(fill=fill, expand=expand, side="bottom")

entryButton = tk.Button(master=frame, 
                        text=f"Enter Entry",
                        font='sans 10',
                        state="disabled")
entryButton.pack(fill=fill, expand=expand)

root.mainloop()

In [9]:
from datetime import timedelta
td = timedelta(milliseconds=10)
td.total_seconds() * 5

0.05